In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [8]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
niter = 500
rank = 80
tau = 5
alpha = 1
gamma = 0.5
rho = 0.5

#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha
#for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#    for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#        for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
            #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#            for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
                #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                for alpha in (0.5,1,1.5,2):
#                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    #print('Compression Ratio', ((1024*28*28+10*1024+(8*(rank)+32*np.square(rank))*2)/(1024*28*28+10*1024+1024*1024*2)), (8*(rank)+32*np.square(rank))*2/(1024*1024*2))
        
print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    d0 = 561 #561 =3*11*17
    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 6 

    W1 = 0.2*init.kaiming_normal_(torch.empty(d1, d0, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    #W1 = 0.01*torch.randn(d1, d0, device=device)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = 0.2*init.kaiming_normal_(torch.empty(d2, d1, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = 0.2*init.kaiming_normal_(torch.empty(d3, d2, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = 0.2*init.kaiming_normal_(torch.empty(d4, d3, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)


    W5 = 0.2*init.kaiming_normal_(torch.empty(d5, d4, device=device), a=0, mode='fan_in', nonlinearity='leaky_relu')
    b5 = 0*torch.ones(d5, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = U5 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V5 = (y_one_hot + gamma*U5 + alpha*V5)/(1 + gamma + alpha)

        # update U4 
        U5 = (gamma*V5 + rho*(torch.mm(W5,V4) + b5.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W5, b5 = updateWb_org(U5,V4,W5,b5,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)



  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b5.repeat(1, N), W5, a4_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        pred_test = torch.argmax(torch.addmm(b5.repeat(1, N_test), W5, a4_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V5,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,U5,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4

    CR_1=((total_variabels)+(d4*d5))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/rank40_100times.csv')
    #new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'seed':seed} 
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/rank40_100times.csv',index=False)


filename= "KaimingNormal_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/4 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 5 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 2.6375598907470703 - sq_loss: 661.6377563476562 - tot_loss: 1252.4216935971053 - acc: 0.1868879216539717 - val_acc: 0.18052256532066507
Repeatition 1 Epoch 2 / 500 
 - time: 2.6039326190948486 - sq_loss: 294.0611877441406 - tot_loss: 515.9870989508927 - acc: 0.176550598476605 - val_acc: 0.166610111978283
Repeatition 1 Epoch 3 / 500 
 - time: 2.5808520317077637 - sq_loss: 157.22232055664062 - tot_loss: 292.56710113584995 - acc: 0.31120783460282914 - val_acc: 0.27315914489311166
Repeatition 1 Epoch 4 / 500 
 - time: 2.5344326496124268 - sq_loss: 84.4797592163086 - tot_loss: 174.0689796321094 - acc: 0.3261697497279652 - val_acc: 0.2982694265354598
Repeatition 1 Epoch 5 / 500 
 - time: 2.59975004196167 - sq_loss: 45.32595443725586 - tot_loss: 108.69854910112917 - acc: 0.3373231773667029 - val_acc: 0.3182897862232779
Repeatition 1 Epoch 6 / 500 
 - time: 2.616335

Repeatition 1 Epoch 48 / 500 
 - time: 2.6036579608917236 - sq_loss: 0.0006358848186209798 - tot_loss: 2.034348521334323 - acc: 0.8826169749727966 - val_acc: 0.8452663725822871
Repeatition 1 Epoch 49 / 500 
 - time: 2.578646421432495 - sq_loss: 0.0006001151632517576 - tot_loss: 1.9570185113880143 - acc: 0.8861534276387377 - val_acc: 0.8466236851034951
Repeatition 1 Epoch 50 / 500 
 - time: 2.61466121673584 - sq_loss: 0.0005667332443408668 - tot_loss: 1.8977470086829271 - acc: 0.8868335146898803 - val_acc: 0.8489989820156091
Repeatition 1 Epoch 51 / 500 
 - time: 2.596435308456421 - sq_loss: 0.0005356532637961209 - tot_loss: 1.8110378948549624 - acc: 0.8890097932535365 - val_acc: 0.8530709195792331
Repeatition 1 Epoch 52 / 500 
 - time: 2.6295573711395264 - sq_loss: 0.0005068217869848013 - tot_loss: 1.753549203491275 - acc: 0.89050598476605 - val_acc: 0.8537495758398371
Repeatition 1 Epoch 53 / 500 
 - time: 2.5843682289123535 - sq_loss: 0.00047994047054089606 - tot_loss: 1.673892008286

Repeatition 1 Epoch 95 / 500 
 - time: 2.5745086669921875 - sq_loss: 8.050382893998176e-05 - tot_loss: 0.5089267531166115 - acc: 0.9532100108813928 - val_acc: 0.9219545300305395
Repeatition 1 Epoch 96 / 500 
 - time: 2.6259751319885254 - sq_loss: 7.793564145686105e-05 - tot_loss: 0.4872952892610556 - acc: 0.954842219804135 - val_acc: 0.9226331862911435
Repeatition 1 Epoch 97 / 500 
 - time: 2.6161415576934814 - sq_loss: 7.551769522251561e-05 - tot_loss: 0.5002871584488275 - acc: 0.9552502720348205 - val_acc: 0.9233118425517476
Repeatition 1 Epoch 98 / 500 
 - time: 2.613579034805298 - sq_loss: 7.314662798307836e-05 - tot_loss: 0.48083477474074243 - acc: 0.956474428726877 - val_acc: 0.9250084832032576
Repeatition 1 Epoch 99 / 500 
 - time: 2.586127996444702 - sq_loss: 7.096209446899593e-05 - tot_loss: 0.46223756508811675 - acc: 0.9567464635473341 - val_acc: 0.9253478113335596
Repeatition 1 Epoch 100 / 500 
 - time: 2.57139253616333 - sq_loss: 6.888387724757195e-05 - tot_loss: 0.46186192

Repeatition 1 Epoch 141 / 500 
 - time: 2.5629851818084717 - sq_loss: 2.903899621742312e-05 - tot_loss: 0.27966554446697955 - acc: 0.9722524483133841 - val_acc: 0.9457074991516796
Repeatition 1 Epoch 142 / 500 
 - time: 2.546632766723633 - sq_loss: 2.8656579161179252e-05 - tot_loss: 0.2696296038656669 - acc: 0.9723884657236126 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 143 / 500 
 - time: 2.56050443649292 - sq_loss: 2.8283837309572846e-05 - tot_loss: 0.256755765406524 - acc: 0.9733405875952121 - val_acc: 0.9467254835425857
Repeatition 1 Epoch 144 / 500 
 - time: 2.5527026653289795 - sq_loss: 2.7910540666198358e-05 - tot_loss: 0.2559852330971353 - acc: 0.9736126224156693 - val_acc: 0.9467254835425857
Repeatition 1 Epoch 145 / 500 
 - time: 2.581336259841919 - sq_loss: 2.7553307518246584e-05 - tot_loss: 0.2540289009592698 - acc: 0.9737486398258978 - val_acc: 0.9467254835425857
Repeatition 1 Epoch 146 / 500 
 - time: 2.526615858078003 - sq_loss: 2.7204645448364317e-05 - tot_loss: 0

Repeatition 1 Epoch 187 / 500 
 - time: 2.496161699295044 - sq_loss: 1.9038865502807312e-05 - tot_loss: 0.2213034903369362 - acc: 0.9783732317736671 - val_acc: 0.9531727180183237
Repeatition 1 Epoch 188 / 500 
 - time: 2.544497489929199 - sq_loss: 1.892045111162588e-05 - tot_loss: 0.21913481493382392 - acc: 0.97810119695321 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 189 / 500 
 - time: 2.530749797821045 - sq_loss: 1.880655690911226e-05 - tot_loss: 0.20523395332250516 - acc: 0.9783732317736671 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 190 / 500 
 - time: 2.5644490718841553 - sq_loss: 1.8695964172366075e-05 - tot_loss: 0.2078518537973082 - acc: 0.9786452665941241 - val_acc: 0.9538513742789277
Repeatition 1 Epoch 191 / 500 
 - time: 2.5676791667938232 - sq_loss: 1.858666837506462e-05 - tot_loss: 0.1866051402002995 - acc: 0.9786452665941241 - val_acc: 0.9538513742789277
Repeatition 1 Epoch 192 / 500 
 - time: 2.5720438957214355 - sq_loss: 1.848075226007495e-05 - tot_loss: 0.

...

Repeatition 1 Epoch 279 / 500 
 - time: 2.5258429050445557 - sq_loss: 1.311063806497259e-05 - tot_loss: 0.1763833865602038 - acc: 0.9834058759521219 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 280 / 500 
 - time: 2.543368339538574 - sq_loss: 1.307168622588506e-05 - tot_loss: 0.16536730741007943 - acc: 0.9835418933623504 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 281 / 500 
 - time: 2.5044772624969482 - sq_loss: 1.3031415619479958e-05 - tot_loss: 0.18074201403848633 - acc: 0.9834058759521219 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 282 / 500 
 - time: 2.5382721424102783 - sq_loss: 1.2990982213523239e-05 - tot_loss: 0.1695560083702503 - acc: 0.9838139281828074 - val_acc: 0.9592806243637597
Repeatition 1 Epoch 283 / 500 
 - time: 2.536607503890991 - sq_loss: 1.2954565136169549e-05 - tot_loss: 0.17630867662707317 - acc: 0.9835418933623504 - val_acc: 0.9589412962334578
Repeatition 1 Epoch 284 / 500 
 - time: 2.518209457397461 - sq_loss: 1.2915124898427166e-05 - tot_los

Repeatition 1 Epoch 325 / 500 
 - time: 2.6256003379821777 - sq_loss: 1.1587598237383645e-05 - tot_loss: 0.16759071316016616 - acc: 0.9849020674646355 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 326 / 500 
 - time: 2.6399340629577637 - sq_loss: 1.1559928680071607e-05 - tot_loss: 0.1710993659636415 - acc: 0.9849020674646355 - val_acc: 0.9609772650152698
Repeatition 1 Epoch 327 / 500 
 - time: 2.569925308227539 - sq_loss: 1.153070024884073e-05 - tot_loss: 0.18062174516337848 - acc: 0.9849020674646355 - val_acc: 0.9606379368849678
Repeatition 1 Epoch 328 / 500 
 - time: 2.548815965652466 - sq_loss: 1.1501771041366737e-05 - tot_loss: 0.15692839881396026 - acc: 0.9849020674646355 - val_acc: 0.9609772650152698
Repeatition 1 Epoch 329 / 500 
 - time: 2.639240026473999 - sq_loss: 1.1471426660136785e-05 - tot_loss: 0.18202931672286127 - acc: 0.985038084874864 - val_acc: 0.9613165931455717
Repeatition 1 Epoch 330 / 500 
 - time: 2.5581326484680176 - sq_loss: 1.144250654760981e-05 - tot_los

Repeatition 1 Epoch 371 / 500 
 - time: 2.545010805130005 - sq_loss: 1.0446728083479684e-05 - tot_loss: 0.17516255293480754 - acc: 0.9863982589771491 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 372 / 500 
 - time: 2.569648027420044 - sq_loss: 1.0422820196254179e-05 - tot_loss: 0.17231780838746147 - acc: 0.9863982589771491 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 373 / 500 
 - time: 2.5541746616363525 - sq_loss: 1.0401165127404965e-05 - tot_loss: 0.1773869645171402 - acc: 0.9862622415669206 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 374 / 500 
 - time: 2.5586299896240234 - sq_loss: 1.037540550896665e-05 - tot_loss: 0.16138160310613614 - acc: 0.9862622415669206 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 375 / 500 
 - time: 2.562387704849243 - sq_loss: 1.0352353456255514e-05 - tot_loss: 0.16990714352152736 - acc: 0.9863982589771491 - val_acc: 0.9623345775364778
Repeatition 1 Epoch 376 / 500 
 - time: 2.5686166286468506 - sq_loss: 1.0330408258596435e-05 - tot_l

Repeatition 1 Epoch 417 / 500 
 - time: 2.621614933013916 - sq_loss: 9.534846867609303e-06 - tot_loss: 0.1760902989734987 - acc: 0.9870783460282916 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 418 / 500 
 - time: 2.573643207550049 - sq_loss: 9.515342753729783e-06 - tot_loss: 0.1904046115948006 - acc: 0.9872143634385201 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 419 / 500 
 - time: 2.595935344696045 - sq_loss: 9.497400242253207e-06 - tot_loss: 0.1627280603328316 - acc: 0.9874863982589771 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 420 / 500 
 - time: 2.6362740993499756 - sq_loss: 9.477842468186282e-06 - tot_loss: 0.17882537340327787 - acc: 0.9870783460282916 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 421 / 500 
 - time: 2.6132609844207764 - sq_loss: 9.459787179366685e-06 - tot_loss: 0.1733666700152554 - acc: 0.9873503808487486 - val_acc: 0.9630132337970818
Repeatition 1 Epoch 422 / 500 
 - time: 2.602483034133911 - sq_loss: 9.442498594580684e-06 - tot_loss: 0.17

Repeatition 1 Epoch 463 / 500 
 - time: 2.5516610145568848 - sq_loss: 8.777322364039719e-06 - tot_loss: 0.17893656226785737 - acc: 0.9881664853101197 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 464 / 500 
 - time: 2.5722155570983887 - sq_loss: 8.762988727539778e-06 - tot_loss: 0.1655837557889086 - acc: 0.9881664853101197 - val_acc: 0.9640312181879878
Repeatition 1 Epoch 465 / 500 
 - time: 2.6039388179779053 - sq_loss: 8.748920663492754e-06 - tot_loss: 0.16275120566189827 - acc: 0.9881664853101197 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 466 / 500 
 - time: 2.573087215423584 - sq_loss: 8.734981747693382e-06 - tot_loss: 0.17598205195572092 - acc: 0.9881664853101197 - val_acc: 0.9633525619273838
Repeatition 1 Epoch 467 / 500 
 - time: 2.5588810443878174 - sq_loss: 8.71986776473932e-06 - tot_loss: 0.18040250776085998 - acc: 0.9880304678998912 - val_acc: 0.9636918900576857
Repeatition 1 Epoch 468 / 500 
 - time: 2.558901786804199 - sq_loss: 8.706784683454316e-06 - tot_loss: 